# Repositorio GCM-TFG
## Problemas de optimización en el modelado de materia oscura galáctica

### `data.py`
Contiene la lista `galaxlist` de galaxias a explorar. Para cada galaxia se abre y lee su respectivo archivo `.arff`, en el que cada fila corresponde a una partícula que se ha observado que gira en torno a la galaxia. Para cada una de estas partículas se tienen los siguientes datos:
* Radio de giro en torno a la galaxia
* Velocidad rotacional de la partícula
* V_{adc}
* Errores
* Velocidad debida a la materia bariónica bariónica

A partir de estos datos construye el diccionario `galaxies` donde se asocia cada galaxia con los datos que se tienen de ella:
* `R`: Vector de radios de giro de las diferentes partículas observadas
* `vrot`: Vector de velocidades rotacionales
* ?
* `errs`: Vector de errores
* `vbary`: Vector de velocidades debidas a la materia bariónica

A continuación se muestra un ejemplo de lectura y recogida de datos de la galaxia DDO43.

In [1]:
from scipy.io import arff
import numpy as np

galaxlist = ["DDO43"]
galaxies = {}
for i in galaxlist:
    fp = open("galaxies/"+i+".arff")
    dt, metadt = arff.loadarff(fp)
    data = []
    for d in dt.tolist():
        data.append(np.asarray(d))
    data = np.asarray(data)
    galaxies[i] = {
        "R": data[:, 0] * 1000,
        "vrot": abs(data[:, 1]),
        "errs": data[:, 3],
        "vbary": np.sqrt(data[:, 4] ** 2 + data[:, 5] ** 2)
    }
    fp.close()

En `data.py` también se declaran las constantes $\nu$ (número de parámetros libres) y $CteDim$ (constante de adimensionalización).
* Puesto que sólo trabajaremos con los perfiles ISO, BUR y NFW, $\nu = 2$.
* $CteDim = \frac{10000}{4.51697\times3.0856776^ 2}$.

### `commonFunctions.py`
Aquí se definen algunas funciones comunes a todas las galaxias y para cualquiera de los perfiles ISO, BUR y NFW.  
* `WeighProd(x, y, sigmas)`: Dados los arrays `x` e `y`, y los pesos `sigmas` devuelve el producto escalar pesado definido en (15).  
* `ginf(x, model)`: Dados un array `x` y un perfil de densidad `model`, devuelve el valor de g cuando s tiende a infinito, definida en la Tabla 2.  
* `eqVLimInf(t, ginf, galaxdata)`: Dados el parámetro `t`, el valor de g definida en la Tabla 2 cuando s tiende a infinito y el diccionario `galaxdata` de datos de la galaxia, devuelve la ecuación definida en (33).  
* `g0(x, model)`: Dado un array `x` y un perfil de densidad `model`, devuelve el valor de g cuando s tiende a cero, definida en la Tabla 2.  
* `eqVLim0(t, g0, galaxdata)`: Dados el parámetro `t`, el valor de g definida en la Tabla 2 cuando s tiende a cero y el diccionario `galaxdata` de datos de la galaxia, devuelve la ecuación definida en (35).  
* `v(r, s, model)`: Dado un array de radios `r`, un array de inversos de parámetros de escalas `s` y un perfil de densidad de materia oscura `model`, devuelve el valor de la ecuación definida en (18) para estos parámetros.  
* `chiquad(rho, s, galaxdata)`: Dados un array de parámetro de densidad central `rho`, un array de de inversos de parámetros de escala `s` y un diccionario de datos de una galaxia `galaxdata`, devuelve el valor de la ecuación definida en (16) para estos parámetros.  
* `rho(s, galaxdata)`: Dados un array de inversos de parámetros de escala `s` y un diccionario de datos de una galaxia `galaxdata`, devuelve el valor de rho estudiado en la Proposición 1.  
* `alphaMV(s, galaxdata)`: Dados un array de inversos de parámetros de escala `s` y un diccionario de datos de una galaxia `galaxdata`, devuelve el valor de la ecuación (24) para estos parámetros.  
* `vv(galaxdata)`: Dado un diccionario de datos de una galaxia `galaxdata`, devuelve el producto escalar pesado de la velocidad rotacional.  
* `vvbary(galaxdata)`: Dado un diccionario de datos de una galaxia `galaxdata`, devuelve el el producto escalar pesado de la velocidad debida a la materia bariónica.  
* `phi(s, galaxdata)`: Dados un array de inversos de parámetros de escala `s` y un diccionario de datos de una galaxia `galaxdata`, devuelve el valor de la función varphi y el valor de la función rho para estos parámetros.

### `calLimits.py`
Aquí se define la función que calcula los límites de varphi en cero y en infinito, usando el Lema 1.  
* `calLimits(galaxdata)`: Dado un diccionario de datos de una galaxia `galaxdata`, devuelve un array con los valores de los límites de varphi en cero y en infinito.  

### `intervalMinim.py`
Aquí se definen las distintas funciones que forman el algoritmo de minimización del intervalo de búsqueda. Se especifica una tolerancia `tol`$=10^{-2}$ y se fija la semilla del random a 1.  
* `inftestElementwise(eval)`: Dado un array de puntos vecinos al candidato a extremo inferior del intervalo `eval`, devuelve dos booleanos. El primero indica si los puntos vecinos de la derecha cumplen la ecuación (40) y el segundo indica si la cumplen los vecinos de la izquierda.  
* `suptestElementwise(eval)`: Dado un array de puntos vecinos al candidato a extremo superior del intervalo `eval`, devuelve dos booleanos. El primero indica si los puntos vecinos de la izquierda cumplen la ecuación (39) y el segundo indica si la cumplen los vecinos de la derecha.  
* `inftestElementsum(eval)`: Dado un array de puntos vecinos al candidato a extremo inferior del intervalo `eval`, devuelve dos booleanos. El primero indica si la suma de los puntos vecinos de la derecha cumplen la ecuación (40) y el segundo indica si la cumple la suma de los vecinos de la izquierda.
* `suptestElementsum(eval)`: Dado un array de puntos vecinos al candidato a extremo superior del intervalo `eval`, devuelve dos booleanos. El primero indica si la suma de los puntos vecinos de la izquierda cumplen la ecuación (39) y el segundo indica si la cumple la suma de los vecinos de la derecha.  
* `infConditions(test1, test2, intervalinf, stop, i)`: Dados un booleano `test1` indicando si los puntos de la derecha (o su suma) cumplen (40), un booleano `test2` indicando si los puntos de la izquierda (o su suma) cumplen (40), un candidato a extremo inferior del intervalo `intervalinf`, un parámetro que controla la condición de parada del algoritmo `stop` y un parámetro `i` que almacena el anterior candidato a extremo inferior, la función decide si el candidato cumple la condición óptima y en qué dirección moverse.  

In [3]:
def infConditions(test1, test2, intervalinf, stop, i):    
    if (not test1) and test2:
        stop = True
        direction = -1
        i = intervalinf
        intervalinf = intervalinf + random.uniform(0.2, 0.6) * direction
    elif (not test1) and (not test2):
        if stop:
            stop = False
        direction = -1
        intervalinf = intervalinf + random.uniform(0.2, 0.3) * direction
    elif test1 and test2:
        if stop:
            direction = 0
            intervalinf = i
        else:
            direction = 1
            intervalinf = intervalinf + random.uniform(0.2, 0.3) * direction
    else:
        if stop:
            stop = False
        direction = -1
        intervalinf = intervalinf + random.uniform(0.2, 0.3) * direction
    return intervalinf, direction, stop, i

* `supConditions(test1, test2, intervalsup, stop, i)`: Dados un booleano `test1` indicando si los puntos de la izquierda (o su suma) cumplen (39), un booleano `test2` indicando si los puntos de la derecha (o su suma) cumplen (39), un candidato a extremo superior del intervalo `intervalsup`, un parámetro que controla la condición de parada del algoritmo `stop` y un parámetro `i` que almacena el anterior candidato a extremo superior, la función decide si el candidato cumple la condición óptima y en qué dirección moverse.  
* `jumpCondition(twoclosevar, varLimdistance, interval, direction, k)`: Dados un booleano `twoclosevar` indicando si los dos últimos candidatos están "cerca", un valor `varLimdistance` indicando a qué distancia está el candidato del valor del límite, un candidato `interval`, una dirección (-1, 0 o 1) `direction` y un contador de la condición de salto `k`, la función devuelve si ha habido salto y, en caso de que sí, cuál es el nuevo candidato.  
* `intervalMin(varphiLim0, varphiLimInf, galaxdata)`: Dados el valor del límite de varphi en cero `varphiLim0`, el valor del límite de varphi en infinito `varphiLimInf` y un diccionario de datos de una galaxia `galaxdata`, la función realiza la minimización del intervalo de búsqueda. Primero busca el extremo inferior que cumple alguna condición satisfactoria y luego el extremo superior, análogamente. Finalmente devuelve los valores propuestos como extremos, el valor mínimo de varphi encontrado en la búsqueda del extremo inferior y el valor mínimo de varphi encontrado en la búsqueda del extremo superior. También puede devolver datos para la elaboración de gráficas.  

A continuación se muestra la parte del código que explora el espacio buscando candidatos a extremo inferior del intervalo. La búsqueda del extremo superior es análoga.

In [ ]:
# PARA EL INTERVALO INFERIOR #

maxiter = 0
direction = -1
intervalinf = -3
k = 0
lastint, rho = phi(np.array([10 ** intervalinf]), galaxdata)
stop = False
i = 0.0
infminphi = 10**4  # Para devolver el mínimo encontrado en la exploración
infminx = 0        # Para devolver el mínimo encontrado en la exploración

while maxiter < 100 and direction != 0 and k < 10:
    maxiter += 1
    # Creo el vecindario de intervalinf
    s = 10**(intervalinf + np.array([-0.2, -0.1, 0.0, 0.1, 0.2]))
    # Evalúo el vecindario de intervalinf
    varphi, rho = phi(s, galaxdata)
    # Almaceno el mínimo valor encontrado hasta el momento
    if min(varphi) < infminphi:
        infminphi = min(varphi)
        pos = (varphi.tolist()).index(infminphi)
        infminx = s[pos]
# Para evaluar las condiciones del candidato actual
eval = abs(varphi - varphiLim0) / varphiLim0
# Estudio la situación de los puntos vecinos
test1, test2 = inftestElementwise(eval)
# Decido movimiento
intervalinf, direction, stop, i = infConditions(test1, test2, intervalinf, stop, i)
# Actualizo valores
var, rho = phi(np.asarray([10 ** intervalinf]), galaxdata)

# Compruebo si estoy en condición de salto
twoclosevar = abs(var - lastint) < tol
varLimdistance = abs(var - varphiLim0)
jump, intervalinf, k = jumpCondition(twoclosevar, varLimdistance, intervalinf, direction, k)
if jump:
    lastint, rho = phi(np.asarray([float(10 ** intervalinf)]), galaxdata)
else:
    lastint = var

A continuación se muestran las líneas de código que implementan la mejora propuesta en la memoria para el algoritmo de minimización del intervalo de búsqueda, y que sustituyen a la línea 25 de arriba, pero en la parte correspondiente de la búsqueda del extremo superior del intervalo.

In [ ]:
if galaxdata["profile"] == 'BUR' or galaxdata["profile"] == 'NFW':
    eval = abs(varphi - varphiLimInf) / (2*varphiLimInf)
else:
    eval = abs(varphi - varphiLimInf) / varphiLimInf

### `varphiMinim.py`
Aquí se definen las distintas funciones que forman el algoritmo de minimización de la función varphi.
* `getIMD(intizq, intder, galaxdata)`: Dados el extremo inferior del intervalo `intizq`, el extremo superior del intervalo `intder`y un diccionario de datos de una galaxia `galaxdata`, la función devuelve el valor medio `m`y su evaluación en varphi, así como un punto aleatorio a la derecha y otro a la izquierda, con sus respectivas evaluaciones en varphi.
* `reductionInterval(varphiLim0, varphiLimInf, intinfmin, intsupmin, intervalinf, intervalsup)`: Realiza la mejora propuesta en la memoria para el algoritmo de minimización de varphi. Dados el límite de varphi en 0 `varphiLim0`, el límite de varphi en infinito `varphiLimInf`, el punto mínimo encontrado en la exploración del intervalo inferior `intinfmin`, el punto mínimo encontrado en la exploración del intervalo superior `intsupmin`, el extremo inferior del intervalo calculado en intervalMinim.py `intervalinf` y el extremo superior del intervalo calculado en intervalMinim.py `intervalsup`, la función devuelve el intervalo de búsqueda nuevamente reducido (en caso de que haya sido posible reducirlo).

In [ ]:
# Si el límite de varphi cuando s tiende a cero es mayor que el límite cuando tiende a infinito,
# comprobamos si el valor mínimo encontrado en la búsqueda del intervalo inferior está más cerca 
# del límite en cero que del límite en infinito. En caso de que sí, podemos tomar la ordenada de 
# ese mínimo encontrado como el nuevo extremo inferior del intervalo. En caso contrario, nos 
# quedamos con el extremo inferior deducido en el algoritmo de minimización del intervalo.
# Si el límite de varphi cuando s tiende a cero es menor que el límite cuando tiende a infinito,
# comprobamos si la distancia entre el límite y el valor mínimo encontrado en la búsqueda del 
# intervalo inferiores menor que 0'1. En caso de que sí, podemos tomar la ordenada de ese 
# mínimo encontrado como el nuevo extremo inferior del intervalo. En caso contrario, nos 
# quedamos con el extremo inferior deducido en el algoritmo de minimización del intervalo.
intinf = intervalinf
if varphiLim0 > varphiLimInf:
    if abs(varphiLim0 - intinfmin[1]) < abs(varphiLimInf - intinfmin[1]):
        intinf = intinfmin[0]
else:
    if abs(varphiLim0 - intinfmin[1]) < 0.1:
        intinf = intinfmin[0]
# Si el límite de varphi cuando s tiende a infinito es mayor que el límite cuando tiende a cero,
# comprobamos si el valor mínimo encontrado en la búsqueda del intervalo superior está más cerca 
# del límite en infinito que del límite en cero. En caso de que sí, podemos tomar la ordenada de 
# ese mínimo encontrado como el nuevo extremo superior del intervalo. En caso contrario, nos 
# quedamos con el extremo superior deducido en el algoritmo de minimización del intervalo.
# Si el límite de varphi cuando s tiende a infinito es menor que el límite cuando tiende a cero,
# comprobamos si la distancia entre el límite y el valor mínimo encontrado en la búsqueda del 
# intervalo superior es menor que 0'1. En caso de que sí, podemos tomar la ordenada de ese 
# mínimo encontrado como el nuevo extremo superior del intervalo. En caso contrario, nos 
# quedamos con el extremo superior deducido en el algoritmo de minimización del intervalo.
intsup = intervalsup
if varphiLimInf > varphiLim0:
    if abs(varphiLimInf - intsupmin[1]) < abs(varphiLim0 - intsupmin[1]):
        intsup = intsupmin[0]
else:
    if abs(varphiLimInf - intsupmin[1]) < 0.1:
        intsup = intsupmin[0]
return intinf, intsup

* `varphiMin(varphiLim0, varphiLimInf, intinfmin, intsupmin, intervalinf, intervalsup, galaxdata)`: Dados el límite de varphi cuando s tiende a 0 `varphiLim0`, el límite de varphi cuando s tiende a infinito `varphiLimInf`, el punto mínimo encontrado en la exploración del intervalo inferior `intinfmin`, el punto mínimo encontrado en la exploración del intervalo superior `intsupmin`, el extremo inferior del intervalo calculado en intervalMinim.py `intervalinf`, el extremo superior del intervalo calculado en intervalMinim.py `intervalsup` y el diccionario de datos de una galaxia `galaxdata`, la función realiza la exploración de varphi y devuelve el mínimo valor encontrado.

In [ ]:
tol = 10**-8
# Mejora propuesta en la memoria
intervalinf, intervalsup = reductionInterval(varphiLim0, varphiLimInf,
                                         intinfmin, intsupmin, intervalinf, intervalsup)
# División del intervalo en escala logarítmica
subint = np.asarray(np.logspace(np.log10(intervalinf), np.log10(intervalsup), 8))
# Para ir actualizando el valor mínimo de varphi
bestphi = 10**4
s = 0
# Control de las situaciones de fork
nfork = 0
cfork = 0
while s < len(subint) - 1:
    # Control de las situaciones de fork: no se permiten más de tres subdivisiones nuevas 
    # por cada uno de los subintervalos iniciales
    if s == nfork + 1:
        nfork = 0
        cfork = 0
    M = 1
    lastM = 0
    # Evalúo los extremos del subintervalo actual en varphi y tomo como valor mínimo inicial
    # el que me devuelva un valor menor
    intizq = subint[s]
    intder = subint[s+1]
    izqphi, rhoi = phi(np.array([intizq]), galaxdata)
    derphi, rhod = phi(np.array([intder]), galaxdata)
    if izqphi < derphi:
        minphi = izqphi
        rho = rhoi
        minphiX = intizq
    else:
        minphi = derphi
        rho = rhod
        minphiX = intder
    # En bucle hasta que los valores del punto medio actual y el punto medio anterior 
    # evaluados en varphi estén demasiado cerca
    while abs(M - lastM) > tol:
        lastM = M
        IMD = getIMD(intizq, intder, galaxdata)
        m = IMD[1][0]
        M = IMD[1][1]
        I = IMD[0][1]
        D = IMD[2][1]
        # Si varphi es menor a la izquierda del punto medio y mayor a la derecha,
        # tomamos el punto medio como nuevo extremo superior del subintervalo actual
        if I < M < D:
            intder = m
        # Si varphi es menor a la derecha del punto medio y mayor a la izquierda,
        # tomamos el punto medio como nuevo extremo inferior del subintervalo actual
        elif I > M > D:
            intizq = m
        # En cualquier otro caso, comprobamos si podemos hacer fork. En caso de que sí, 
        # introducimos un nuevo subintervalo a la lista de subintervalos. En caso de que
        # no escogemos de forma aleatoria si el punto medio es el nuevo extremo inferior 
        # o el nuevo extremo superior
        else:
            if nfork + 1 > 3:
                if random.randint(0, 1):
                    intder = m
                else:
                    intizq = m
            else:
                intder = m
                subint = np.insert(subint, s+1, m)
                nfork += 1
        # Actualizamos el mínimo encontrado 
        sorted_IMD = sorted(IMD, key=lambda tup: tup[1])
        if sorted_IMD[0][1] < minphi:
            minphi = sorted_IMD[0][1]
            rho = sorted_IMD[0][2]
            minphiX = sorted_IMD[0][0]
    if minphi < bestphi:
        bestphi = minphi
        bestrho = rho
        bestphiX = minphiX
    s += 1
    cfork += 1

### `redMethRotCurveFitting.py`